In [1]:
import pandas as pd
import time as time
import requests

In [2]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [3]:
# This function scrapes at least min_posts from Reddit not including [removed] 
# and [deleted] posts, and contingent on whether a post is_self or not

def scrape_reddit(subreddit, min_posts, is_self):
     
    url = 'https://api.pushshift.io/reddit/search/submission'
    df = pd.DataFrame()

    while len(df) < min_posts:
        try:
            if is_self == 'Y':
                if len(df) == 0:
                    params = {
                        'subreddit': subreddit,
                        'size': 100,
                        'before': int(time.time()),
                        'is_self': True,
                    }
                    res = requests.get(url, params)
                    data = res.json()
                    posts = data['data']
                    df = pd.DataFrame(posts)
                    df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
                    df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)
                else:
                    params = {
                        'subreddit': subreddit,
                        'size': 100,
                        'before': df.iloc[-1]['created_utc'],
                        'is_self': True,
                    }
                    res = requests.get(url, params)
                    data = res.json()
                    posts = data['data']
                    df2 = pd.DataFrame(posts)
                    df = df.append(df2, ignore_index=True)
                    df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
                    df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)        
            elif is_self == 'N':
                if len(df) == 0:
                    params = {
                        'subreddit': subreddit,
                        'size': 100,
                        'before': int(time.time()),
                    }
                    res = requests.get(url, params)
                    data = res.json()
                    posts = data['data']
                    df = pd.DataFrame(posts)
                    df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
                    df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)
                else:
                    params = {
                        'subreddit': subreddit,
                        'size': 100,
                        'before': df.iloc[-1]['created_utc'],
                    }
                    res = requests.get(url, params)
                    data = res.json()
                    posts = data['data']
                    df2 = pd.DataFrame(posts)
                    df = df.append(df2, ignore_index=True)
                    df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
                    df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)
            else:
                print(f"Enter 'Y' or 'N' for third argument.")
                break
        except JSONDecodeError:
            pass
        except:
            print('Other error.')
            
        time.sleep(3)
        print(len(df))
    df.reset_index(inplace=True)
    return df

In [4]:
# CHANGE THIS CELL FOR PULLING NEW INFORMATION

subreddit = 'stockmarket'

In [5]:
df = scrape_reddit(subreddit, 10_000, 'Y')

56
108
154
202
255
300
361
421
481
536
597
651
705
757
817
867
921
975
1027
1086
1145
1189
1243
1292
1352
1405
1462
1512
1561
1607
1660
1712
1767
1824
1879
1936
1987
2035
2083
2129
2179
2227
2273
2317
2371
2415
2465
2517
2574
2625
2680
2736
2791
2844
2892
2947
3000
3047
3096
3153
3204
3255
3302
3358
3417
3460
3507
3573
3622
3675
3721
3766
3815
3863
3920
3966
4018
4073
4129
4181
4233
4287
4333
4395
4450
4510
4564
4618
4671
4732
4775
4825
4890
4954
5016
5068
5127
5184
5231
5289
5346
5406
5464
5518
5569
5627
5683
5745
5796
5842
5895
5950
6009
6062
6126
6173
6243
6308
6368
6431
6494
6558
6616
6673
6725
6791
6853
6914
6967
7025
7082
7144
7196
7254
7306
7354
7397
7460
7509
7581
7662
7748
7832
7906
7993
8069
8147
8222
8303
8388
8473
8557
8630
8709
8793
8876
8966
9048
9124
9200
9281
9361
9445
9525
9603
9683
9753
9831
9912
9992
10067


In [6]:
df[['title', 'selftext', 'author', 'subreddit']].to_csv(f'./{subreddit}.csv', index=False)